## Домашняя работа 2

Аксенова Анна БКЛ182

Проверяем pep-8

In [ ]:
# Почему-то у меня все перестает работать с проверкой пепа
# Но можно попробовать раскомментить
# %load_ext pycodestyle_magic
# %pycodestyle_on

Импортируем *json* (он нужен для всей работы) и *Mystem* (для первого задания)

In [1]:
import json
from pymystem3 import Mystem

Я буду работать с романом Даниила Гранина "Иду на грозу"

Он оочень крутой, по нему снят [фильм](https://www.kinopoisk.ru/film/42573/) с хорошей оценкой на кинопоиске.

Считываем текст из файла:


In [2]:
with open("idu-na-grozu.txt", encoding='utf-8') as fh:
    text = fh.read()

Очищаем его от пунктуации (punctuation из string мне кажется недостаточной):

In [3]:
punctuation = (list(""".,…;:!”?$%^&*№()_—=+|[]{}\"/<>`~±§«»°1234567890""")
               + ['- ', ' -', ' - ', " '", "' "])
for char in punctuation:
    text = text.strip().replace(char, '')

In [4]:
m = Mystem()

**Задание 1**

Считаем время работы mystem.analyse и записываем результат в файл *ing_mystem.json* в соответствующем формате.

In [5]:
%%time 
an = m.analyze(text)

CPU times: user 2.33 s, sys: 279 ms, total: 2.61 s
Wall time: 11.1 s


In [6]:
with open("ing_mystem.json", "w", encoding='utf-8') as fh:
    json.dump(an, fh, ensure_ascii=False)

**Задание 2**

Импортируем библиотеки и модули.

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

Считаем время работы программы по обработке текста

In [8]:
%%time

tokens = [w.lower() for w in word_tokenize(text) if w.isalpha()]
# Выбираем все строки из букв,
# приводим их к нижнему регистру и токенизируем
morph = MorphAnalyzer()
morph_tokens = []
# Делаем список из самых вероятных морфологических характеристик токенов
for token in tokens:
    first = morph.parse(token)[0]
    morph_tokens.append(first)

CPU times: user 3.93 s, sys: 92.4 ms, total: 4.02 s
Wall time: 4.11 s


In [9]:
print(len(morph_tokens)) # просто интересно

97138


In [10]:
lemmas = []
# Для каждого токена находим лемму и часть речи
for token in morph_tokens:
    lemm = (token.word, (token.normal_form, token.tag.POS))
    lemmas.append(lemm)
# Записываем результат в файл
with open("ing_tokens.json", "w", encoding='utf-8') as fh:
    json.dump(lemmas, fh, ensure_ascii=False)

**Задание 4**


Распечатаем все части речи с их частотностью в тексте, не убирая стоп-слова

In [16]:
from collections import Counter
POS = []
for lemma in lemmas:
    #  Если убрать комментарии ниже, то мы поймем,
    #  что как None определены аббревиатуры и английские слова
    #  if lemma[1] == None:
    #      print(lemma)
    POS.append(lemma[1][1])
POS = Counter(POS)
for key, value in POS.items():
    print(key, value / len(lemmas))

None 0.00035001750087504374
ADJF 0.0933517264098499
NOUN 0.24699911466161542
VERB 0.16385966357141388
PREP 0.0889147398546398
NUMR 0.0026971936832135726
NPRO 0.10346105540571146
PRTF 0.0070827070765302974
CONJ 0.0965018839177253
PRCL 0.05874117235273529
GRND 0.009831373921637259
ADVB 0.06196339228726142
INFN 0.035269410529349995
ADJS 0.01230208569251992
COMP 0.00481788795322119
PRED 0.007720974284008318
PRTS 0.0038090139801107703
INTJ 0.002326586917581173


Теперь, чтобы получить содержательный результат, уберем стоп-слова.

Найдем самые частотные глаголы и наречия.

In [33]:
from nltk.corpus import stopwords

sw = stopwords.words('russian')
filtered = [l[1] for l in lemmas if l[1][0] not in sw]

[('libru', None), ('электронный', 'ADJF'), ('библиотека', 'NOUN'), ('название', 'NOUN'), ('книга', 'NOUN'), ('идти', 'VERB'), ('гроза', 'NOUN'), ('автор', 'NOUN'), ('гранин', 'NOUN'), ('даниил', 'NOUN'), ('жанр', 'NOUN'), ('современный', 'ADJF'), ('проза', 'NOUN'), ('адрес', 'NOUN'), ('книга', 'NOUN'), ('часть', 'NOUN'), ('волшебник', 'NOUN'), ('прилететь', 'VERB'), ('москва', 'NOUN'), ('шесть', 'ADJF'), ('май', 'NOUN'), ('восемь', 'NUMR'), ('часы', 'NOUN'), ('утро', 'NOUN'), ('сбежать', 'VERB'), ('качаться', 'PRTF'), ('трап', 'NOUN'), ('бетонный', 'ADJF'), ('плита', 'NOUN'), ('аэродром', 'NOUN'), ('взгляд', 'NOUN'), ('встречать', 'PRTF'), ('устремляться', 'VERB'), ('соскальзывать', 'VERB'), ('никто', 'NPRO'), ('находить', 'VERB'), ('ничто', 'NPRO'), ('особенный', 'ADJF'), ('это', 'NPRO'), ('стройный', 'ADJF'), ('загорелый', 'ADJF'), ('парень', 'NOUN'), ('модный', 'ADJF'), ('ворсистый', 'ADJF'), ('пиджакеон', 'NOUN'), ('пройти', 'VERB'), ('сквозь', 'PREP'), ('толпа', 'NOUN'), ('оставля

In [34]:
verbs = []
adverbs = []
for lemma in filtered:
    if lemma[1] == "VERB":
        verbs.append(lemma[0])
    elif lemma[1] == "ADVB":
        adverbs.append(lemma[0])
verbs = Counter(verbs).most_common()
adverbs = Counter(adverbs).most_common()

In [35]:
print('Топ самых частотных глаголов')
for i in range(20):
    print(i+1, *verbs[i])

Топ самых частотных глаголов
1 сказать 647
2 мочь 544
3 хотеть 202
4 знать 201
5 стать 156
6 спросить 151
7 думать 123
8 говорить 113
9 смотреть 104
10 видеть 95
11 понимать 90
12 посмотреть 87
13 подумать 84
14 идти 83
15 чувствовать 78
16 иметь 74
17 взять 69
18 считать 68
19 сидеть 67
20 понять 67


In [36]:
print('Топ самых частотных наречий')
for i in range(20):
    print(i+1, *adverbs[i])

Топ самых частотных наречий
1 почему 139
2 несколько 72
3 снова 69
4 пока 69
5 слишком 65
6 быстро 48
7 вместе 48
8 вообще 44
9 сразу 43
10 иначе 40
11 наверное 39
12 давно 37
13 очень 34
14 долго 32
15 сегодня 29
16 невозможно 27
17 впервые 27
18 мало 26
19 откуда 26
20 ясно 24


**Задание 5**

Посчитаем самые частотные n-граммы

In [37]:
# переопределим список filtered,
# т.к. теперь нам не важна частиречная принадлежность
filtered = [f[0] for f in filtered]

In [38]:
bigrm = nltk.bigrams(filtered)
popular = Counter(bigrm).most_common()
print('Самые частотные биграммы:')
for i in range(20):
    print(*popular[i][0], '\t\t', popular[i][1])

Самые частотные биграммы:
сказать крылов 		 97
сказать тулина 		 57
весь это 		 55
весь равно 		 51
сергей илья 		 37
друг друг 		 31
это весь 		 30
вера матвей 		 27
сказать агат 		 26
яков иван 		 26
аркадий борис 		 25
человек который 		 23
сказать женя 		 20
мочь это 		 18
крылов это 		 18
сей пора 		 18
никак мочь 		 18
это ещё 		 17
крылов весь 		 16
сказать голицын 		 15


In [39]:
trigrm = nltk.trigrams(filtered)
popular = Counter(trigrm).most_common()
print('Самые частотные триграммы:')
for i in range(20):
    print(*popular[i][0], '\t\t', popular[i][1])

Самые частотные триграммы:
это иметь значение 		 5
сказать крылов это 		 5
начальник отдел кадр 		 5
никак мочь понять 		 4
весь знать знать 		 3
чепуха сказать крылов 		 3
весь равно ничто 		 3
ничто мочь поделать 		 3
мощный кучевой облако 		 3
сказать тулина ничто 		 3
сергей илья сказать 		 3
это весь равно 		 3
вынуть записной книжка 		 3
сунуть рука карман 		 3
весь равно это 		 3
делать свой дело 		 3
сказать крылов дать 		 3
человек который мочь 		 3
закон сохранение энергия 		 3
лопаться мыльный пузырь 		 3


Как мы видим, самые частотные биграмммы и триграммы включают в себя имена главных героев, а также устойчивые словосочетания типа *делать свое дело*, *это имеет значение*, *сия пора*. Также мы видим словосочетание *мощный кучевой облако*, это отражает содержание книги, так как сюжет завязан на изучении грозовых облаков группой ученых физиков.